In [1]:
import pandas as pd

In [2]:
origin_data = pd.read_csv('california_3938_2016_minute_data_modify_carbon.csv')
origin_data.head()

,Year,Month,Day,Hour,Minute,Downwelling Global Solar [W/m2],Outdoor Drybulb Temperature [C],Wind Speed [m/s],Wind Direction [deg from N],Day Type,Equipment Electric Power [kW],Electricity Pricing [$/kWh],Occupancy,Carbon Intensity [kg/kWh]
0,2016,6,1,0,0,-1.2,12.0,2.2,225.0,3,0.607,0.03025,1.0,0.088517
1,2016,6,1,0,1,-1.1,12.0,2.0,225.0,3,0.241,0.03025,1.0,0.088517
2,2016,6,1,0,2,-1.1,11.9,1.8,226.0,3,0.710,0.03025,1.0,0.088517
3,2016,6,1,0,3,-1.1,11.9,1.7,222.0,3,0.183,0.03025,1.0,0.088517
4,2016,6,1,0,4,-1.0,11.9,1.6,223.0,3,0.726,0.03025,1.0,0.088517


In [ ]:
"""
添加洗衣需求：

1. 每一天洗衣服需求的概率不一样，根据星期几来决定
2. 如果当天需求洗衣服，需要根据时间和房屋占用状态来决定具体洗衣需求时间
3. 决定了具体洗衣需求时间后，通过正态分布决定可等待时间
4. 根据以上规则，在原始数据集上添加两行，一是当前是否有洗衣需求（没有为0，有的话为1）
"""